# Park Behavioral Profiling & Structural Diagnosis

This notebook reproduces the analysis for:
1. Behavioral Clustering (K-Means, k=5)
2. Predictive Modeling (LOOCV)
3. Model Interpretation (SHAP, PDP)
4. Structural Diagnosis (Efficiency Index)

## 1. Behavioral Clustering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set paths
BASE_DIR = r"c:\Users\jour\Documents\GitHub\accessibility_park-1\park-behavioral-clustering"
DATA_DIR = os.path.join(BASE_DIR, "data", "processed")
RESULTS_TABLES_DIR = os.path.join(BASE_DIR, "results", "tables")
RESULTS_FIGURES_DIR = os.path.join(BASE_DIR, "results", "figures")
SRC_DIR = os.path.join(BASE_DIR, "src")

os.makedirs(RESULTS_TABLES_DIR, exist_ok=True)
os.makedirs(RESULTS_FIGURES_DIR, exist_ok=True)
os.makedirs(SRC_DIR, exist_ok=True)

# 1. Load Data
clusters_path = os.path.join(DATA_DIR, "enhanced_behavioral_clusters.csv")
features_path = os.path.join(RESULTS_TABLES_DIR, "Table_S1_Clustering_Features.csv")

df = pd.read_csv(clusters_path)
features_df = pd.read_csv(features_path)

# Extract feature names
feature_names = features_df['Feature Name'].tolist()
print(f"Loaded {len(feature_names)} features for clustering.")

# 2. Preprocess
X = df[feature_names]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. K-Means Clustering (k=5)
k = 5
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_scaled)

# 4. Evaluation Metrics & Plot
# Elbow & Silhouette
inertias = []
silhouettes = []
k_range = range(2, 10)

for n_k in k_range:
    km = KMeans(n_clusters=n_k, random_state=42, n_init=10)
    km_labels = km.fit_predict(X_scaled)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(X_scaled, km_labels))

fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:red'
ax1.set_xlabel('Number of clusters (k)')
ax1.set_ylabel('Inertia', color=color)
ax1.plot(k_range, inertias, marker='o', color=color, label='Inertia')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Silhouette Score', color=color)
ax2.plot(k_range, silhouettes, marker='s', color=color, label='Silhouette')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('K-Means Optimization: Elbow Method & Silhouette Score')
fig.tight_layout()
plt.savefig(os.path.join(RESULTS_FIGURES_DIR, 'Figure_1_Clustering_Optimization.png'))
plt.close()

# 5. Update Dataset
df['cluster'] = labels

# Save updated dataset
df.to_csv(clusters_path, index=False)
print(f"Updated clusters saved to {clusters_path}")

# 6. Generate Cluster Description Table
# Calculate mean of features per cluster
cluster_summary = df.groupby('cluster')[feature_names].mean().T
cluster_summary.columns = [f'Cluster {i}' for i in range(k)]
cluster_summary.to_csv(os.path.join(RESULTS_TABLES_DIR, 'Table_4_Cluster_Descriptions.csv'))
print("Cluster description table saved.")


## 2. Predictive Modeling (LOOCV)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
import os

# Set paths
BASE_DIR = r"c:\Users\jour\Documents\GitHub\accessibility_park-1\park-behavioral-clustering"
DATA_DIR = os.path.join(BASE_DIR, "data", "processed")
RESULTS_TABLES_DIR = os.path.join(BASE_DIR, "results", "tables")
SRC_DIR = os.path.join(BASE_DIR, "src")

# 1. Load Data
clusters_path = os.path.join(DATA_DIR, "enhanced_behavioral_clusters.csv")
dataset_path = os.path.join(DATA_DIR, "final_dataset_18parks.csv")
features_path = os.path.join(RESULTS_TABLES_DIR, "Table_S2_Predictive_Features.csv")

df_clusters = pd.read_csv(clusters_path)
df_data = pd.read_csv(dataset_path)
df_features = pd.read_csv(features_path)

# Merge: enhanced_clusters has 'cluster', final_dataset has 'park_name' and features
# Note: final_dataset ALSO has a 'cluster' column which might be old. We should drop it or use the one from enhanced_clusters.
# enhanced_clusters has 'park_name' as index? No, let's check. 
# Step 14 output shows 'park_name' is NOT a column, it seems to be in the first column but unnamed in header?
# Wait, look at Step 14 output: "Gangseo Hangang Park" is the first value. The header line is:
# ",avg_ppltn,max_ppltn..." -> The first column name is empty string or space.
# I need to handle this.

# Reload with specific handling
df_clusters = pd.read_csv(clusters_path)
if df_clusters.columns[0].strip() == '' or 'Unnamed' in df_clusters.columns[0]:
    df_clusters.rename(columns={df_clusters.columns[0]: 'park_name'}, inplace=True)

# Dataset 18 parks might be fine.
df_data = pd.read_csv(dataset_path)

# Merge
# We want 'cluster' from df_clusters and FEATURES from df_data.
merged_df = pd.merge(df_clusters[['park_name', 'cluster']], df_data, on='park_name', suffixes=('_target', '_data'))

# Use the cluster from df_clusters as target (it deals with the re-run k=5)
target_col = 'cluster_target'

# Get Predictive Features
feature_list = df_features['Feature Name'].tolist()
# Filter out any that might be missing
available_features = [f for f in feature_list if f in merged_df.columns]
missing_features = [f for f in feature_list if f not in merged_df.columns]
if missing_features:
    print(f"Warning: Missing features from dataset: {missing_features}")

X = merged_df[available_features]
y = merged_df[target_col]
park_names = merged_df['park_name']

print(f"Data Loaded. Shape: {X.shape}. Target distribution:\n{y.value_counts()}")

# 2. Predictive Modeling with LOOCV
models = {
    'Logistic Regression': LogisticRegression(max_iter=2000, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

results = []
loo = LeaveOneOut()

print("\nStarting LOOCV...")

for name, model in models.items():
    y_true_all = []
    y_pred_all = []
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if name == 'XGBoost':
            # XGBoost requires 0..N-1 labels. 
            # If a class is missing in y_train, we need to remap.
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            y_train_enc = le.fit_transform(y_train)
            
            # Check if we have enough classes
            if len(np.unique(y_train_enc)) < 2:
                 pred = [y_train.iloc[0]] # Fallback
            else:
                model.fit(X_train, y_train_enc)
                pred_enc = model.predict(X_test)
                # Map back. Note: if model predicts a class, it's an index in le.classes_
                # But wait, if X_test leads to a prediction, it gives an integer.
                # We need to inverse transform.
                pred = le.inverse_transform(pred_enc)
        
        else:
            # Standard Sklearn models
            if len(y_train.unique()) < 2:
                pred = [y_train.iloc[0]] 
            else:
                model.fit(X_train, y_train)
                pred = model.predict(X_test)
        
        y_true_all.append(y_test.values[0])
        y_pred_all.append(pred[0])
        
    acc = accuracy_score(y_true_all, y_pred_all)
    f1 = f1_score(y_true_all, y_pred_all, average='macro')
    
    print(f"Model: {name} | Accuracy: {acc:.4f} | F1-Macro: {f1:.4f}")
    
    results.append({
        'Model': name,
        'Accuracy': acc,
        'F1-Score (Macro)': f1
    })

# 3. Save Results
results_df = pd.DataFrame(results)
output_path = os.path.join(RESULTS_TABLES_DIR, "Table_6_Model_Performance.csv")
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path}")


## 3. Interpretation & Diagnosis

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import PartialDependenceDisplay
import os

# Set paths
BASE_DIR = r"c:\Users\jour\Documents\GitHub\accessibility_park-1\park-behavioral-clustering"
DATA_DIR = os.path.join(BASE_DIR, "data", "processed")
RESULTS_TABLES_DIR = os.path.join(BASE_DIR, "results", "tables")
RESULTS_FIGURES_DIR = os.path.join(BASE_DIR, "results", "figures")

# 1. Load Data
clusters_path = os.path.join(DATA_DIR, "enhanced_behavioral_clusters.csv")
dataset_path = os.path.join(DATA_DIR, "final_dataset_18parks.csv")
features_path = os.path.join(RESULTS_TABLES_DIR, "Table_S2_Predictive_Features.csv")

df_clusters = pd.read_csv(clusters_path)
if df_clusters.columns[0].strip() == '' or 'Unnamed' in df_clusters.columns[0]:
    df_clusters.rename(columns={df_clusters.columns[0]: 'park_name'}, inplace=True)

df_data = pd.read_csv(dataset_path)
df_features = pd.read_csv(features_path)

# Merge
# Need 'avg_ppltn' for Volume target (Efficiency Index)
merged_df = pd.merge(df_clusters[['park_name', 'cluster', 'avg_ppltn']], df_data, on='park_name', suffixes=('_target', '_data'))

feature_list = df_features['Feature Name'].tolist()
available_features = [f for f in feature_list if f in merged_df.columns]

X = merged_df[available_features]
y_cls = merged_df['cluster_target']
y_vol = np.log1p(merged_df['avg_ppltn']) # Target for regression

print(f"Data Loaded. X Shape: {X.shape}")

# ---------------------------------------------------------
# Part 1: Interpretation (XGBoost Classifier)
# ---------------------------------------------------------
print("Running Interpretation (SHAP & PDP)...")

# Encode labels
le = LabelEncoder()
y_cls_enc = le.fit_transform(y_cls)

# Train Classifier
model_cls = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
model_cls.fit(X, y_cls_enc)

# SHAP
# Explainer
explainer = shap.TreeExplainer(model_cls)
shap_values = explainer.shap_values(X)

# Summary Plot
plt.figure(figsize=(10, 8))
# For multiclass, shap_values is a list of arrays (one for each class).
# Summary plot for multiclass sums them up or we can pick one?
# Usually 'shap.summary_plot(shap_values, X)' handles multiclass by stacking or via color.
shap.summary_plot(shap_values, X, show=False)
plt.savefig(os.path.join(RESULTS_FIGURES_DIR, 'Figure_2_SHAP_Summary.png'), bbox_inches='tight')
plt.close()

# PDP
# Get Top 3 features by importance
importances = model_cls.feature_importances_
indices = np.argsort(importances)[::-1]
top_features = X.columns[indices[:3]].tolist()
print(f"Top features for PDP: {top_features}")

# Create PDP
# Note: multiclass PDP usually requires specifying the target class. 
# We'll plot for each class or just average?
# Sklearn's PartialDependenceDisplay allows 'target' param. 
# We'll generate PDP for class 0 (Evening Urban) as example, or iterate?
# Let's plot for the most populous class or just default (which might average for binary, or require target for multi).
# For XGBoost estimator in sklearn wrapper, we can use sklearn's display.
# We will generate one figure with subplots for the top features.
# We have to pick a target class for multiclass PDP. Let's pick all classes in one plot per feature? Too messy.
# Let's pick the 'Evening Urban' (Cluster 0) and 'Mega' (Cluster 3 or so). 
# Actually, let's just do it for Class 0 for now as an example, or loop through top features.
# If we don't specify target for multiclass, it might fail.
# Let's assume Class 0 is interesting.
target_class_idx = 0 
fig, ax = plt.subplots(figsize=(12, 4))
PartialDependenceDisplay.from_estimator(model_cls, X, top_features, target=target_class_idx, ax=ax)
plt.suptitle(f"Partial Dependence Plots (Target: Class {le.inverse_transform([target_class_idx])[0]})")
plt.savefig(os.path.join(RESULTS_FIGURES_DIR, 'Figure_3_PDP.png'), bbox_inches='tight')
plt.close()


# ---------------------------------------------------------
# Part 2: Diagnosis (XGBoost Regressor)
# ---------------------------------------------------------
print("Running Structural Diagnosis (Efficiency Index)...")

model_reg = xgb.XGBRegressor(random_state=42)
model_reg.fit(X, y_vol)
y_pred_vol = model_reg.predict(X)

# Calculate Efficiency
residuals = y_vol - y_pred_vol
# Efficiency Index: Let's use Residual directly. Positive = More efficient than expected.

merged_df['log_actual'] = y_vol
merged_df['log_predicted'] = y_pred_vol
merged_df['efficiency_residual'] = residuals

# Define Outliers
mean_res = residuals.mean()
std_res = residuals.std()
threshold = 1.0 * std_res # Using 1.0 std for small sample to highlight SOME outliers

merged_df['efficiency_status'] = 'Normal'
merged_df.loc[merged_df['efficiency_residual'] > mean_res + threshold, 'efficiency_status'] = 'Positive Outlier'
merged_df.loc[merged_df['efficiency_residual'] < mean_res - threshold, 'efficiency_status'] = 'Negative Outlier'

# Save Table
out_table = merged_df[['park_name', 'cluster_target', 'avg_ppltn', 'log_actual', 'log_predicted', 'efficiency_residual', 'efficiency_status']]
out_table.to_csv(os.path.join(RESULTS_TABLES_DIR, 'Table_Efficiency_Diagnosis.csv'), index=False)

# Plot Diagnosis
plt.figure(figsize=(10, 8))
sns.scatterplot(data=merged_df, x='log_predicted', y='log_actual', hue='efficiency_status', style='cluster_target', s=100)
# Add diagonal line
min_val = min(y_vol.min(), y_pred_vol.min())
max_val = max(y_vol.max(), y_pred_vol.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.5)

# Label outliers
for i, row in merged_df.iterrows():
    if row['efficiency_status'] != 'Normal':
        plt.text(row['log_predicted'], row['log_actual'], row['park_name'], fontsize=9)

plt.title('Structural Diagnosis: Actual vs Predicted Volume')
plt.xlabel('Predicted Log Volume (Potential)')
plt.ylabel('Actual Log Volume (Usage)')
plt.savefig(os.path.join(RESULTS_FIGURES_DIR, 'Figure_4_Efficiency_Diagnosis.png'))
plt.close()

print("Interpretation and Diagnosis Complete.")
